# Financial Data Analysis: Stock Performance & Revenue Visualization

![Financial Analysis Dashboard](https://images.unsplash.com/photo-1611974789855-9c2a0a7236a3?w=1100&h=400&fit=crop&auto=format)

## Project Overview

A comprehensive data analysis project that combines API integration and web scraping to extract, process, and visualize financial data for comparative stock analysis. The project demonstrates end-to-end data pipeline development for investment decision support.

## Key Features

- **Automated data extraction** from Yahoo Finance API (yfinance)
- **Web scraping implementation** for quarterly revenue data
- **Interactive dashboards** with dual-axis visualization
- **Comparative analysis** of Tesla (TSLA) vs GameStop (GME)

## Technical Stack

- **Python** | **Pandas** | **BeautifulSoup** | **Plotly** | **yfinance**

## Deliverables

- Historical stock price extraction and processing
- Revenue data collection via web scraping
- Interactive multi-panel visualizations
- Correlation analysis between stock performance and financial metrics

## Business Value

Provides automated financial data aggregation and visualization tools for investment analysis, demonstrating proficiency in data engineering, API integration, and financial data visualization techniques.

## Environment Setup & Dependencies

In [131]:
# Install required packages for financial data analysis and visualization
!pip install yfinance      
!pip install bs4           
!pip install nbformat      
!pip install --upgrade plotly

In [132]:
# Data extraction and analysis libraries
import yfinance as yf                      # Yahoo Finance API
import pandas as pd                        # Data manipulation
import requests                            # HTTP requests for web scraping
from bs4 import BeautifulSoup              # HTML parsing

# Visualization libraries  
import plotly.graph_objects as go          # Interactive plotting
from plotly.subplots import make_subplots  # Multi-panel charts
import plotly.io as pio

pio.renderers.default = "iframe"           # Jupyter notebook compatibility

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Helper Functions

In [133]:
# def make_graph(stock_data, revenue_data, stock):
#     """
#     Create interactive dual-axis chart showing stock price and revenue data
    
#     Parameters:
#     stock_data (DataFrame): Stock price data with Date and Close columns
#     revenue_data (DataFrame): Revenue data with Date and Revenue columns  
#     stock (str): Title for the chart
#     """
#     # Create dual-panel chart layout with shared x-axis
#     fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
#                         subplot_titles=("Historical Share Price", "Historical Revenue"), 
#                         vertical_spacing=.3)
    
#     # Filter data to specified date ranges
#     stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
#     revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    
#     # Add stock price trace
#     fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), 
#                             y=stock_data_specific.Close.astype("float"), 
#                             name="Share Price"), row=1, col=1)
    
#     # Add revenue trace 
#     fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), 
#                             y=revenue_data_specific.Revenue.astype("float"), 
#                             name="Revenue"), row=2, col=1)
    
#     # Update axes and layout
#     fig.update_xaxes(title_text="Date", row=1, col=1)
#     fig.update_xaxes(title_text="Date", row=2, col=1)
#     fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
#     fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
#     fig.update_layout(showlegend=False, height=900, title=stock, xaxis_rangeslider_visible=True)
    
#     # Display interactive chart
#     fig.show()
    
#     # Return figure for potential export
#     return fig

In [134]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    # from IPython.display import display, HTML
    # fig_html = fig.to_html()
    # display(HTML(fig_html))

In [135]:
def find_revenue_table_index(tables, keywords):
    """
    Find the index of table containing specified keywords
    
    Parameters:
    tables (list): List of pandas DataFrames from pd.read_html()
    keywords (list): List of keywords to search for in table content
    
    Returns:
    int: Index of the table containing all specified keywords
    """
    for i, table in enumerate(tables):
        table_text = table.to_string()
        # Check if all keywords are present in the table
        if all(keyword in table_text for keyword in keywords):
            return i
    return 0  # Default to first table if no match found

## Use yfinance to Extract Stock Data

In [136]:
# Create Tesla ticker object and display basic information
tesla = yf.Ticker("TSLA")
print(tesla)

yfinance.Ticker object <TSLA>


In [137]:
# Extract historical stock data for Tesla and display first 5 rows
tesla_data = tesla.history(period="max")
tesla_data.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [138]:
# Reset index to convert Date from index to column and verify data structure
tesla_data.reset_index(inplace=True)
tesla_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [139]:
# Comprehensive data overview for quality assessment
print(f"Dataset dimensions: {tesla_data.shape[0]} rows × {tesla_data.shape[1]} columns")
print(f"Date coverage: {tesla_data['Date'].min().strftime('%Y-%m-%d')} to {tesla_data['Date'].max().strftime('%Y-%m-%d')}")
print(f"Trading days captured: {len(tesla_data)} days")
print(f"Missing values: {tesla_data.isnull().sum().sum()}")

Dataset dimensions: 3804 rows × 8 columns
Date coverage: 2010-06-29 to 2025-08-12
Trading days captured: 3804 days
Missing values: 0


## Use Webscraping to Extract Tesla Revenue Data

In [140]:
# Define URL containing revenue data for web scraping
tesla_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

# Download webpage content using requests and store HTML as text
tesla_response = requests.get(tesla_url)
tesla_html_data  = tesla_response.text

# Verify successful request and display content summary
print(f"Status Code: {tesla_response.status_code}")
print(f"Content Length: {len(tesla_html_data):,} characters")
print(f"Content Type: {tesla_response.headers.get('content-type', 'Unknown')}")

Status Code: 200
Content Length: 64,779 characters
Content Type: text/html


In [141]:
# Extract and clean Tesla revenue data with correct column names
tesla_tables = pd.read_html(tesla_html_data)
tesla_keywords = ["Tesla", "Quarterly", "Revenue"]
tesla_index = find_revenue_table_index(tesla_tables, tesla_keywords)
tesla_revenue = tesla_tables[tesla_index]
tesla_revenue.head()

,Tesla Quarterly Revenue (Millions of US $),Tesla Quarterly Revenue (Millions of US $).1
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [142]:
# Rename columns to standard format
tesla_revenue.columns = ['Date', 'Revenue']
tesla_revenue.head()

,Date,Revenue
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [143]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(r',|\$', "", regex=True)
tesla_revenue.head()

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757


In [144]:
# # Clean revenue column - remove $ and commas
# tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$', "")
# tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',', "")

# # Convert Revenue to numeric format for analysis
# tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors='coerce')

# tesla_revenue.head()

In [145]:
# Remove null values and empty strings
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [146]:
# Display latest Tesla revenue entries
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## Use yfinance to Extract Stock Data

In [147]:
# Create GameStop ticker object for data extraction
gme = yf.Ticker("GME")

In [148]:
# Extract historical stock data for GameStop and display first 5 rows
gme_data = gme.history(period="max")
gme_data.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [149]:
# Reset index to convert Date from index to column and verify data structure
gme_data.reset_index(inplace=True)

In [150]:
# Comprehensive data overview for quality assessment
print(f"Dataset dimensions: {gme_data.shape[0]} rows × {gme_data.shape[1]} columns")
print(f"Date coverage: {gme_data['Date'].min().strftime('%Y-%m-%d')} to {gme_data['Date'].max().strftime('%Y-%m-%d')}")
print(f"Trading days captured: {len(gme_data)} days")
print(f"Missing values: {gme_data.isnull().sum().sum()}")

Dataset dimensions: 5912 rows × 8 columns
Date coverage: 2002-02-13 to 2025-08-12
Trading days captured: 5912 days
Missing values: 0


In [151]:
# Display first 5 rows of GameStop stock data
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Use Webscraping to Extract GME Revenue Data

In [152]:
# Define URL containing GameStop revenue data for web scraping
gme_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

# Download webpage content using requests and store HTML as text
gme_response = requests.get(gme_url)
gme_html_data = gme_response.text

# Verify successful request and display content summary
print(f"Status Code: {gme_response.status_code}")
print(f"Content Length: {len(gme_html_data):,} characters")
print(f"Content Type: {gme_response.headers.get('content-type', 'Unknown')}")

Status Code: 200
Content Length: 88,703 characters
Content Type: text/html


In [153]:
# Extract and clean GameStop revenue data
gme_tables= pd.read_html(gme_html_data)

# Find GameStop revenue table using keyword matching
gme_keywords = ["GameStop", "Revenue"]
gme_index = find_revenue_table_index(gme_tables, gme_keywords)
gme_revenue = gme_tables[gme_index]
gme_revenue.head()

,GameStop Annual Revenue (Millions of US $),GameStop Annual Revenue (Millions of US $).1
0,2020,"$6,466"
1,2019,"$8,285"
2,2018,"$8,547"
3,2017,"$7,965"
4,2016,"$9,364"


In [154]:
# Rename columns to standard format
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(r',|\$', "", regex=True)
gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"].astype(str) + "-12-31")
tesla_revenue.head()
gme_revenue.head()

,Date,Revenue
0,2020-12-31,6466
1,2019-12-31,8285
2,2018-12-31,8547
3,2017-12-31,7965
4,2016-12-31,9364


In [155]:
# # Clean revenue column - remove $ and commas
# gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace('$', "")
# gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',', "")

# # Convert Revenue to numeric format for analysis
# gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors='coerce')

# # Convert year format to proper datetime (GameStop data contains years only)
# gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"].astype(str) + "-12-31")
# gme_revenue.head()

In [156]:
# Remove null values and empty strings
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [157]:
# Display latest GameStop revenue entries
gme_revenue.tail()

,Date,Revenue
11,2009-12-31,8806
12,2008-12-31,7094
13,2007-12-31,5319
14,2006-12-31,3092
15,2005-12-31,1843


## Plot Tesla Stock Graph

In [158]:
# Generate Tesla stock price and revenue visualization
make_graph(tesla_data, tesla_revenue, "Tesla")

## Plot GameStop Stock Graph

In [159]:
make_graph(gme_data, gme_revenue, 'GameStop')